In [2]:
import numpy as np
import h5py
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0, 4.0)
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

np.random.seed(1)

def zero_pad(X, pad):
    
    X_pad = np.pad(X,((0,0), (pad, pad), (pad, pad), (0,0)), 'constant')
    
    return X_pad

In [3]:
np.random.seed(1)
x = np.random.rand(4, 3, 3, 2)
x_pad = zero_pad(x, 2)
print(x.shape)
print(x_pad.shape)
print(x[1,1])
print(x_pad[1,1])


(4, 3, 3, 2)
(4, 7, 7, 2)
[[0.87638915 0.89460666]
 [0.08504421 0.03905478]
 [0.16983042 0.8781425 ]]
[[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]


In [4]:
def conv_single_step(a_slice_prev, W, b):

    s = a_slice_prev * W
    Z = np.sum(s)
    Z = Z + b
    
    return Z

In [5]:
np.random.seed(1)
a_slice_prev = np.random.rand(4, 4, 3)
W = np.random.randn(4,4,3)
b = np.random.randn(1,1,1)

Z = conv_single_step(a_slice_prev, W, b)
print(Z)

[[[2.57809319]]]


In [6]:
def conv_forward(A_prev, W, b, hparameters):
    
    (m, n_H_prev, n_W_prev, n_C_prev) = A_prev.shape
    
    (f, f, n_C_prev, n_C) = W.shape
    
    stride = hparameters['stride']
    pad = hparameters['pad']
    
    n_H = int((n_H_prev - f + 2*pad) / stride + 1)
    n_W = int((n_W_prev - f + 2*pad) / stride + 1)
    
    Z = np.zeros((m, n_H, n_W, n_C))
    
    A_prev_pad = zero_pad(A_prev, pad)
    
    for i in range(m):
        a_prev_pad = A_prev_pad[i, :, :, :]
        for h in range(n_H):
            for w in range(n_W):
                for c in range(n_C):
                    
                    vert_start = stride * h
                    vert_end = vert_start + f
                    horize_start = stride * w
                    horize_end = horize_start + f
                    
                    a_slice_prev = a_prev_pad[vert_start:vert_end, horize_start:horize_end, :]
                    
                    Z[i, h, w, c] = conv_single_step(a_slice_prev, W[:, :, :, c], b[:, :, :, c])
                    
                    assert(Z.shape) == (m, n_H, n_W, n_C)
                    
                    cache = (A_prev, W, b, hparameters)
                    
                    return Z, cache
            
    

In [7]:
np.random.seed(1)
A_prev = np.random.rand(10, 4, 4, 3)
W = np.random.randn(2, 2, 3, 8)
b = np.random.randn(1,1,1,8)
hparameters = {'pad':2,
               'stride':2}

z, cache = conv_forward(A_prev, W, b, hparameters)
print(np.mean(Z))
print(z[3,2,1])
print(cache[0][1][2][3])

2.578093188905089
[0. 0. 0. 0. 0. 0. 0. 0.]
[0.62367221 0.75094243 0.34889834]


In [16]:
def pool_forward(A_prev, hparameters, mode='max'):

    (m, n_H_prev, n_W_prev, n_C_prev) = A_prev.shape

    f = hparameters['f']
    stride = hparameters['stride']

    n_H = int(1+(n_H_prev - f) / stride)
    n_W = int(1+(n_W_prev - f) / stride)
    n_C = n_C_prev

    A = np.zeros((m, n_H, n_W, n_C))

    for i in range(m):
        for h in range(n_H):
            for w in range(n_W):
                for c in range(n_C):
                    vert_start = h * stride
                    vert_end = vert_start + f
                    horiz_start = w * stride
                    horiz_end = horiz_start + f

                    a_prev_slice = A_prev[i, vert_start:vert_end, horiz_start:horiz_end, c]

                    if mode == 'max':
                        A[i, h, w, c] = np.max(a_prev_slice)
                    if mode == 'average':
                        A[i, h, w, c] = np.mean(a_prev_slice)

    cache = (A_prev, hparameters)

    assert(A.shape == (m, n_H, n_W, n_C))

    return A, cache

In [17]:

# def pool_forward(A_prev, hparameters, mode = "max"):
#     """
#     Implements the forward pass of the pooling layer
# 
#     Arguments:
#     A_prev -- Input data, numpy array of shape (m, n_H_prev, n_W_prev, n_C_prev)
#     hparameters -- python dictionary containing "f" and "stride"
#     mode -- the pooling mode you would like to use, defined as a string ("max" or "average")
# 
#     Returns:
#     A -- output of the pool layer, a numpy array of shape (m, n_H, n_W, n_C)
#     cache -- cache used in the backward pass of the pooling layer, contains the input and hparameters 
#     """
# 
#     # Retrieve dimensions from the input shape
#     (m, n_H_prev, n_W_prev, n_C_prev) = A_prev.shape
# 
#     # Retrieve hyperparameters from "hparameters"
#     f = hparameters["f"]
#     stride = hparameters["stride"]
# 
#     # Define the dimensions of the output
#     n_H = int(1 + (n_H_prev - f) / stride)
#     n_W = int(1 + (n_W_prev - f) / stride)
#     n_C = n_C_prev
# 
#     # Initialize output matrix A
#     A = np.zeros((m, n_H, n_W, n_C))              
# 
#     ### START CODE HERE ###
#     for i in range(m):                         # loop over the training examples
#         for h in range(n_H):                     # loop on the vertical axis of the output volume
#             for w in range(n_W):                 # loop on the horizontal axis of the output volume
#                 for c in range (n_C):            # loop over the channels of the output volume
# 
#                     # Find the corners of the current "slice" (≈4 lines)
#                     vert_start = h * stride
#                     vert_end = vert_start + f
#                     horiz_start = w * stride
#                     horiz_end = horiz_start + f
# 
#                     # Use the corners to define the current slice on the ith training example of A_prev, channel c. (≈1 line)
#                     a_prev_slice = A_prev[i, vert_start:vert_end, horiz_start:horiz_end, c]
# 
#                     # Compute the pooling operation on the slice. Use an if statment to differentiate the modes. Use np.max/np.mean.
#                     if mode == "max":
#                         A[i, h, w, c] = np.max(a_prev_slice)
#                     elif mode == "average":
#                         A[i, h, w, c] = np.mean(a_prev_slice)
# 
#     ### END CODE HERE ###
# 
#     # Store the input and hparameters in "cache" for pool_backward()
#     cache = (A_prev, hparameters)
# 
#     # Making sure your output shape is correct
#     assert(A.shape == (m, n_H, n_W, n_C))
# 
#     return A, cache


In [18]:
np.random.seed(1)
A_prev = np.random.randn(2, 4, 4, 3)
hparameters = {"stride" : 2, "f": 3}

A, cache = pool_forward(A_prev, hparameters)
print("mode = max")
print("A =", A)
print()
A, cache = pool_forward(A_prev, hparameters, mode = "average")
print("mode = average")
print("A =", A)


mode = max
A = [[[[1.74481176 0.86540763 1.13376944]]]


 [[[1.13162939 1.51981682 2.18557541]]]]

mode = average
A = [[[[ 0.02105773 -0.20328806 -0.40389855]]]


 [[[-0.22154621  0.51716526  0.48155844]]]]
